# Batch Process Inferences

In [2]:
from utils import VideoReaders, DetectorLoader
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import segmentation

In [3]:
model = DetectorLoader.YOLACT('./weights/yolact_resnet50_54_800000.pth', threshold = 0.11)

/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))


In [25]:
relevant_videos = ['0029', '0030', '0031', '0032', '0033', '0034', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042']

for video in relevant_videos:
    
    frame_provider = VideoReaders.VideoReader('../datasets/slip-fall/fall-database/{}.mp4'.format(video))
    video_length, shape = frame_provider.properties()
    
    output = []
    frames = []
    boundaries = []

    for frame in frame_provider:
        frames.append(frame)
        c, s, bb, ma = model.predict(frame)
        idx = np.where(c == 0)
        pixelwise_arrays = []
        for item in idx:
            for n, qq in enumerate(item):
                pixelwise = np.zeros_like(ma[qq,...])
                pixelwise = ma[qq,...].astype(np.uint8)
                pixelwise[np.where(pixelwise == 1)] = n + 1
                pixelwise_arrays.append(pixelwise)

        filtered = []
        for array in pixelwise_arrays:
            if np.max(array) != 0:
                filtered.append(array)

        stacked = np.sum(filtered, axis = 0)
        output.append(stacked)

    for im, mask in zip(frames, output):
        boundaries.append(segmentation.mark_boundaries(im, mask, mode = 'thick', color = (1,0,0)))


    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter("./results/{}.mp4".format(video), 0x7634706d, 5.0, max([x.shape for x in boundaries])[:-1])
    for capture in boundaries:
        out_capture = (capture * 255).astype(np.uint8)
        writer.write(capture)
    writer.release()


(432, 768)


error: OpenCV(4.5.4) /tmp/pip-req-build-3129w7z7/opencv/modules/videoio/src/cap_ffmpeg.cpp:186: error: (-215:Assertion failed) image.depth() == CV_8U in function 'write'
